In [100]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# # stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [78]:

acc=[]
for i in open('/home/devanshparmar/devansh/accounts3','r').readlines():
    acc.append(i.rstrip('\n'))

# fac=[16,253,21,22,28,37,40,38,39,245,201,389,380,219,220,221,205,203,251,258
# ]



In [79]:
# acc=['660003246401'
# ]

In [80]:
len(acc)

24570

In [81]:
type_id=[1563054,1563000,1564044,1564165,1563384,1564042,1564157,1564299]

In [261]:
# type_id

In [119]:
df_all=[]

# for f_id in type_id:
query ='''
SELECT DISTINCT
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    dpd.id AS DPD_ID,
    dm.path AS filePath,
    dm.id as document_id,
    em.id as encounter_id
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON dm.account_id = em.account_id
        AND dm.service_date = em.service_date
        AND dm.physician_id = em.physician_id
        AND em.is_active = 1
WHERE
    is_physician_document = 1
        AND dm.is_active = 1
        and account_number in ({})
        and dpd.id = (select MAX(id)FROM document_processing_detail
          WHERE document_id = dm.id)

ORDER BY dm.created_date DESC;
'''.format(','.join(['"'+str(x)+'"' for x in acc]))
# print(query)

# for f in fac:

# query='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         left JOIN
#     document_code dc ON dc.document_id = dm.id
#         left JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dc.code_category = 'EANDM'
#         and is_physician_document=1
#         and dm.is_active=1
#         and account_number in ({})
#         and dpd.id = (SELECT MAX(id)
#           FROM document_processing_detail
#           WHERE document_id = dm.id)
#     order by dm.created_date desc;
# '''.format(','.join(str(x) for x in acc))
           
df_all.append(pd.read_sql(query, con=db_connection))
print(len(df_all))


1


In [120]:
df_query=pd.concat(df_all)

In [121]:
df_query

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_request_s3_path,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,DPD_ID,filePath,document_id,encounter_id
0,87391091,ezcapc/SCP/data/120936/nlp/cdp-xml/20230216/70...,ezcapc/SCP/data/120936/nlp/ner/20230216/709910...,83,2023-02-16 06:00:00,ezcapc/SCP/data/120936/nlp/pcs-xml/20230216/70...,ezcapc/SCP/data/120936/preprocessing/20230216/...,ezcapc/SCP/data/120936/preprocessing/20230216/...,SCP/data/120936/document/20230216/628538098.pdf,24564,...,ezcapc/SCP/data/120936/eandmcs/20230216/679391...,ezcapc/SCP/data/120936/eandmcs/20230216/679391...,ezcapc/SCP/data/120936/ruleengine/20230216/395...,ezcapc/SCP/data/120936/cptcs/20230216/7084772_...,ezcapc/SCP/data/120936/cptcs/20230216/7084772_...,6582919,7197566,SCP/data/120936/document/20230216/628538098_20...,34213565,5316010
1,87391091,ezcapc/SCP/data/120936/nlp/cdp-xml/20230216/70...,ezcapc/SCP/data/120936/nlp/ner/20230216/709907...,83,2023-02-16 06:00:00,ezcapc/SCP/data/120936/nlp/pcs-xml/20230216/70...,ezcapc/SCP/data/120936/preprocessing/20230216/...,ezcapc/SCP/data/120936/preprocessing/20230216/...,SCP/data/120936/document/20230216/628538073.pdf,24564,...,ezcapc/SCP/data/120936/eandmcs/20230216/679387...,ezcapc/SCP/data/120936/eandmcs/20230216/679387...,ezcapc/SCP/data/120936/ruleengine/20230216/395...,ezcapc/SCP/data/120936/cptcs/20230216/7084727_...,ezcapc/SCP/data/120936/cptcs/20230216/7084727_...,6582919,7197517,SCP/data/120936/document/20230216/628538073_20...,34213338,5316010
2,87177232,ezcapc/SCP/data/2464/nlp/cdp-xml/20230203/7006...,ezcapc/SCP/data/2464/nlp/ner/20230203/7006253_...,365,2023-02-03 06:00:00,ezcapc/SCP/data/2464/nlp/pcs-xml/20230203/7006...,ezcapc/SCP/data/2464/preprocessing/20230203/70...,ezcapc/SCP/data/2464/preprocessing/20230203/70...,SCP/data/2464/document/20230203/627932579.pdf,1083080,...,ezcapc/SCP/data/2464/eandmcs/20230203/6705194_...,ezcapc/SCP/data/2464/eandmcs/20230203/6705194_...,ezcapc/SCP/data/2464/ruleengine/20230203/38651...,ezcapc/SCP/data/2464/cptcs/20230203/6992064_62...,ezcapc/SCP/data/2464/cptcs/20230203/6992064_62...,6160638,7105274,SCP/data/2464/document/20230203/627932579_2023...,33744514,5047473
3,87080966,ezcapc/SCP/data/2464/nlp/cdp-xml/20230130/7006...,ezcapc/SCP/data/2464/nlp/ner/20230130/7006251_...,365,2023-01-30 06:00:00,ezcapc/SCP/data/2464/nlp/pcs-xml/20230130/7006...,ezcapc/SCP/data/2464/preprocessing/20230130/70...,ezcapc/SCP/data/2464/preprocessing/20230130/70...,SCP/data/2464/document/20230130/627932421.pdf,1083080,...,ezcapc/SCP/data/2464/eandmcs/20230130/6705192_...,ezcapc/SCP/data/2464/eandmcs/20230130/6705192_...,ezcapc/SCP/data/2464/ruleengine/20230130/38651...,ezcapc/SCP/data/2464/cptcs/20230130/6992062_62...,ezcapc/SCP/data/2464/cptcs/20230130/6992062_62...,6017053,7105273,SCP/data/2464/document/20230130/627932421_2023...,33744513,4920036
4,87080966,ezcapc/SCP/data/2464/nlp/cdp-xml/20230130/7006...,ezcapc/SCP/data/2464/nlp/ner/20230130/7006250_...,365,2023-01-30 06:00:00,ezcapc/SCP/data/2464/nlp/pcs-xml/20230130/7006...,ezcapc/SCP/data/2464/preprocessing/20230130/70...,ezcapc/SCP/data/2464/preprocessing/20230130/70...,SCP/data/2464/document/20230130/627932504.pdf,1083080,...,ezcapc/SCP/data/2464/eandmcs/20230130/6705190_...,ezcapc/SCP/data/2464/eandmcs/20230130/6705190_...,ezcapc/SCP/data/2464/ruleengine/20230130/38651...,ezcapc/SCP/data/2464/cptcs/20230130/6992060_62...,ezcapc/SCP/data/2464/cptcs/20230130/6992060_62...,6017053,7105271,SCP/data/2464/document/20230130/627932504_2023...,33744510,4920036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25705,84612425,ezcapc/SCP/data/7198/nlp/cdp-xml/20220907/3060...,ezcapc/SCP/data/7198/nlp/ner/20220907/3060324_...,72,2022-09-07 05:00:00,ezcapc/SCP/data/7198/nl

In [122]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [123]:
df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id']).first().reset_index()

In [124]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [125]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id', 'DPD_ID',
       'document_id', 'encounter_id'],
      dtype='object')

In [126]:
print(df_query.groupby('facility_id').size().to_markdown())

|   facility_id |    0 |
|--------------:|-----:|
|            49 | 1484 |
|            50 | 2885 |
|            59 | 1373 |
|            60 |  585 |
|            61 | 2113 |
|            64 |  557 |
|            66 | 1516 |
|            67 | 1030 |
|            69 | 1079 |
|            70 | 1396 |
|            71 |  720 |
|            72 | 2446 |
|            73 |  834 |
|            77 |  618 |
|            78 |  696 |
|            82 |  532 |
|            83 |  250 |
|            84 |  789 |
|            85 | 1407 |
|            88 |  953 |
|           365 |  761 |
|           381 | 1686 |


In [127]:
# commands

In [141]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id, dpd_id,t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             # df_query.suggested_code, df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_pcs.xml".format(env, pcs, fac_name,enc_id, acc, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_eandm_request.json".format(env, em_req,fac_name, enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((em_req, '{}/{}/{}_{}_eandm_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, dpd_id,t_id))

        cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name,dpd_id, enc_id,acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.pdf".format(env,ori_path,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_nlp_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_nlp_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [170]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("/home/devanshparmar/devansh/Download/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [171]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [172]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [174]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

0
[49]
1
128
0
[50]
1
114
0
[59]
1
72
0
[60]
1
104
0
[61]
1
108
0
[64]
1
88
0
[66]
1
96
0
[67]
1
88
0
[69]
1
136
0
[70]
1
112
0
[71]
1
74
0
[72]
1
116
0
[73]
1
88
0
[77]
1
80
0
[78]
1
76
0
[82]
1
88
0
[83]
1
70
0
[84]
1
90
0
[85]
1
80
0
[88]
1
74
0
[365]
1
54
0
[381]
1
108


""


In [139]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id', 'DPD_ID',
       'document_id', 'encounter_id'],
      dtype='object')

In [ ]:
# df_query[df_query.account_number.str.contains('0996')]

In [ ]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()